In [1]:
import openferro as of
from openferro.interaction import *


In [2]:
latt = of.BravaisLattice3D(3, 3, 3)
pto = of.System(latt, pbc=True)
dipole_field = pto.add_field(name="dipole", ftype="scalar", value=1)
dipole_field.get_field()

Array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)

In [3]:
pto.get_field_by_name('dipole')

double_well = self_energy_R1_onsite
pto.add_self_interaction('double_well', field_name="dipole", energy_engine=double_well, parameters={'k2': 1, 'alpha': 1, 'offset': 0})

ising_couple = self_energy_R1_neighbor_pbc
pto.add_self_interaction('ising_couple', field_name="dipole", energy_engine=ising_couple, parameters={'J1': 1, 'J2': 1, 'J3': 1, 'offset': 0})


In [6]:
pto.calculate_energy_by_name('double_well')
# pto.calculate_energy_by_name('ising_couple')

Array(54., dtype=float32)